In [14]:
import netCDF4 as nc
import pandas as pd
import numpy as np

### Purpose

Grabbing hourly weather forecast data for 2019 for Florida, Bergen from MET THREDDS: https://thredds.met.no/thredds/catalog.html

### Define netCDF file names by specifying date and time

In [4]:
thredds_urls = list()

for nc_dt in pd.date_range(start='2019-01-01 00:00', end='2019-12-31 23:00', freq='60H'):
    current_url = 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/' + str(nc_dt.month).zfill(2) + '/' + \
        str(nc_dt.day).zfill(2) + '/meps_mbr0_pp_2_5km_2019' + str(nc_dt.month).zfill(2) + str(nc_dt.day).zfill(2) + \
        'T' + str(nc_dt.hour).zfill(2) + 'Z.nc'
    thredds_urls.append(current_url)

In [5]:
thredds_urls[:3]

['https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/01/01/meps_mbr0_pp_2_5km_20190101T00Z.nc',
 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/01/03/meps_mbr0_pp_2_5km_20190103T12Z.nc',
 'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/01/06/meps_mbr0_pp_2_5km_20190106T00Z.nc']

### Find closest weather model point

In [20]:
# Modified code from https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude

from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    closest_point = min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))
    closest_idx = np.argmin([distance(v['lat'],v['lon'],p['lat'],p['lon']) for p in data])
    return closest_point, closest_idx

In [21]:
weather_station = {'lat': 60.383, 'lon': 5.3327} # Bergen, FLorida

In [22]:
nc_data = nc.Dataset(thredds_urls[0])
lats = nc_data.variables['latitude'][:]
lons = nc_data.variables['longitude'][:]

In [23]:
lat_lon_model_points = list()

for lat, lon in zip(lats.flatten(), lons.flatten()):
    lat_lon_model_points.append({'lat': lat, 'lon': lon})

In [24]:
lat_lon_model_points[0:5]

[{'lat': 52.114260619314265, 'lon': 0.506769728067554},
 {'lat': 52.11919772795616, 'lon': 0.5418808356841226},
 {'lat': 52.12412313250235, 'lon': 0.5770007145517959},
 {'lat': 52.12903682801575, 'lon': 0.6121293469988841},
 {'lat': 52.13393880956891, 'lon': 0.6472667153239783}]

In [40]:
closest_point, closest_idx = closest(lat_lon_model_points, weather_station)

In [42]:
print('Closest point:', closest_point, 
      '\nClosest index:', closest_idx)

Closest point: {'lat': 60.37710907397123, 'lon': 5.330741772326141} 
Closest index: 296516


In [43]:
print('Shape of lat and lon arrays:', lats.shape, lons.shape)

Shape of lat and lon arrays: (929, 869) (929, 869)


In [44]:
closest_row_idx = closest_idx // 869
closest_col_idx = closest_idx - (closest_row_idx * 869)

In [45]:
print(lats[closest_row_idx, closest_col_idx], lons[closest_row_idx, closest_col_idx])

60.37710907397123 5.330741772326141


In [54]:
# Check that model point is consistent throughout period (2019)
for url in thredds_urls:
    try:
        nc_data = nc.Dataset(url)
        lat = nc_data.variables['latitude'][closest_row_idx, closest_col_idx]
        lon = nc_data.variables['longitude'][closest_row_idx, closest_col_idx]

        if (lat != 60.37710907397123) | (lon != 5.330741772326141):
            print('Inconsistent weather model point found:', lat, lon)
            
    except OSError as e:
        print(e)

[Errno -90] NetCDF: file not found: b'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/03/02/meps_mbr0_pp_2_5km_20190302T00Z.nc'
[Errno -90] NetCDF: file not found: b'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/03/04/meps_mbr0_pp_2_5km_20190304T12Z.nc'
[Errno -90] NetCDF: file not found: b'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/08/26/meps_mbr0_pp_2_5km_20190826T12Z.nc'


### Get first 60 hours of temperature forecasts for every file

In [133]:
df_list = list()

for url in thredds_urls:
    try:
        nc_data = nc.Dataset(url)
        
        forecast_ref_time = pd.to_datetime([nc_data['forecast_reference_time'][:].tolist()] * 60, unit='s')
        time = pd.to_datetime(nc_data['time'][0:60].tolist(), unit='s')
        temp_kelvin = nc_data.variables['air_temperature_2m'][0:60, 0, closest_row_idx, closest_col_idx].tolist()
        temp_celsius = pd.Series([temp - 273.15 for temp in temp_kelvin])
        
        df_list.append(pd.DataFrame({'forecast_ref_time_utc': forecast_ref_time,
                                     'datetime_utc': time,
                                     'air_temperature_2m': temp_celsius}))
    except OSError as e:
        print(e)

[Errno -90] NetCDF: file not found: b'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/03/02/meps_mbr0_pp_2_5km_20190302T00Z.nc'
[Errno -90] NetCDF: file not found: b'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/03/04/meps_mbr0_pp_2_5km_20190304T12Z.nc'
[Errno -90] NetCDF: file not found: b'https://thredds.met.no/thredds/dodsC/meps25epsarchive/2019/08/26/meps_mbr0_pp_2_5km_20190826T12Z.nc'


In [134]:
df_weather_forecast = pd.concat(df_list, axis=0, ignore_index=True)

In [135]:
df_weather_forecast.head()

,forecast_ref_time_utc,datetime_utc,air_temperature_2m
0,2019-01-01,2019-01-01 00:00:00,5.605432
1,2019-01-01,2019-01-01 01:00:00,5.326746
2,2019-01-01,2019-01-01 02:00:00,5.567499
3,2019-01-01,2019-01-01 03:00:00,5.486780
4,2019-01-01,2019-01-01 04:00:00,5.475427


In [136]:
df_weather_forecast = df_weather_forecast.set_index('datetime_utc').reindex(pd.date_range(start='2019-01-01 00:00', 
                                                                                 end='2019-12-31 23:00', freq='H'))
df_weather_forecast.reset_index(inplace=True, drop=False)

In [137]:
df_weather_forecast[df_weather_forecast.air_temperature_2m.isna()].head()

,index,forecast_ref_time_utc,air_temperature_2m
1440,2019-03-02 00:00:00,NaT,NaN
1441,2019-03-02 01:00:00,NaT,NaN
1442,2019-03-02 02:00:00,NaT,NaN
1443,2019-03-02 03:00:00,NaT,NaN
1444,2019-03-02 04:00:00,NaT,NaN


In [138]:
for idx, _ in df_weather_forecast[df_weather_forecast.air_temperature_2m.isna()].iterrows():
    df_weather_forecast.at[idx, 'air_temperature_2m'] = df_weather_forecast.at[idx - 24, 'air_temperature_2m']

In [139]:
df_weather_forecast.iloc[1440:1445]

,index,forecast_ref_time_utc,air_temperature_2m
1440,2019-03-02 00:00:00,NaT,0.560693
1441,2019-03-02 01:00:00,NaT,0.493280
1442,2019-03-02 02:00:00,NaT,0.251398
1443,2019-03-02 03:00:00,NaT,-0.004980
1444,2019-03-02 04:00:00,NaT,-0.095557


In [140]:
df_weather_forecast.rename({'index': 'datetime_utc'}, axis=1, inplace=True)

In [141]:
df_weather_forecast.head()

,datetime_utc,forecast_ref_time_utc,air_temperature_2m
0,2019-01-01 00:00:00,2019-01-01,5.605432
1,2019-01-01 01:00:00,2019-01-01,5.326746
2,2019-01-01 02:00:00,2019-01-01,5.567499
3,2019-01-01 03:00:00,2019-01-01,5.486780
4,2019-01-01 04:00:00,2019-01-01,5.475427


In [109]:
os.listdir('../../data/weather/florida/csv')

['.ipynb_checkpoints',
 'weather_florida_2013.csv',
 'weather_florida_2014.csv',
 'weather_florida_2015.csv',
 'weather_florida_2016.csv',
 'weather_florida_2017.csv',
 'weather_florida_2018.csv',
 'weather_florida_2019.csv',
 'weather_florida_2020.csv']

In [142]:
df_weather_forecast.to_csv('../../data/weather/florida/csv/weather_forecast_florida_2019.csv', index=False)